In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error as MAE
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
import numpy as np
from gensim.models import word2vec
from gensim.corpora.dictionary import Dictionary
import pickle
from keras.preprocessing import sequence

import time, datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
import gc
from sklearn.model_selection import GridSearchCV
import jieba, pdb
from sklearn.decomposition import PCA


jieba.set_dictionary('jieba_dict/dict.txt.big')
# load stopwords set
stopword_set = set()
with open('jieba_dict/stopwords.txt','r', encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))

model = word2vec.Word2Vec.load("word2vec2.model")


def create_dictionaries(p_model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(p_model.wv.vocab.keys(), allow_update=True)
    w2indx = {v: k + 1 for k, v in gensim_dict.items()}  # 词语的索引，从1开始编号
    w2vec = {word: model[word] for word in w2indx.keys()}  # 词语的词向量
    return w2indx, w2vec

def Convert_orderid(x):
    return str(x).strip('\n')

def Convert_Date(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    date1 = pd.to_datetime(Year+'-'+Month+'-'+Day)
    return date1

def Date2Ticks(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    date1 = str(Year+'/'+Month+'/'+Day)
    return time.mktime(datetime.datetime.strptime(date1, "%Y/%m/%d").timetuple())

index_dict, word_vectors= create_dictionaries(model)
output = open("wordwmbedding.pkl", 'wb')
pickle.dump(index_dict, output)  # 索引字典
pickle.dump(word_vectors, output)  # 词向量字典
output.close()


n_symbols = len(index_dict) + 1  # 索引数字的个数，因为有的词语索引为0，所以+1
embedding_weights = np.zeros((n_symbols, 100))  # 创建l一个n_symbols * 100的0矩阵
for w, index in index_dict.items():  # 从索引为1的词语开始，用词向量填充矩阵
    embedding_weights[index, :] = word_vectors[w]  # 词向量矩阵，第一行是0向量（没有索引为0的词语，未被填充）


C:\Users\Jess\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\Jess\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [2]:
# load dataset
df_order = pd.read_csv("dataset/order.csv")
df_group = pd.read_csv("dataset/group.csv")
df_airline = pd.read_csv("dataset/airline.csv")
df_day_schedule = pd.read_csv("day_schedule_processed.txt")
df_train = pd.read_csv("training-set.csv")
df_test = pd.read_csv("val.csv")
df_result = pd.read_csv("testing-set.csv")
# date Conversion

month = {'Jan': '01', 'Feb': '02' , 'Mar':'03' ,'Apr': '04', 
'May': '05', 'Jun': '06' , 'Jul': '07' , 'Aug':'08', 
'Sep':'09', 'Oct':'10' , 'Nov':'11', 'Dec':'12' }

# group data
df_group['Begin_Date']=df_group.begin_date.apply(lambda x: Convert_Date(x))
df_group['Begin_Tick']=df_group.begin_date.apply(lambda x: Date2Ticks(x))
df_group['SubLine']= df_group.sub_line.apply(lambda x: int(x[14:]))
df_group['Area']= df_group.area.apply(lambda x: int(x[11:]))
df_group['name']= df_group.area.apply(lambda x: len(x))
df_group['group_id']=df_group.group_id.apply(lambda x: Convert_orderid(x))
df_airline['group_id']=df_airline.group_id.apply(lambda x: Convert_orderid(x))
df_order['group_id']=df_order.group_id.apply(lambda x: Convert_orderid(x))
df_day_schedule['group_id']=df_day_schedule.group_id.apply(lambda x: Convert_orderid(x))


group_used_cols=['group_id','Begin_Date','Begin_Tick','days','Area','SubLine','price', 'name']
df_train['order_id']=df_train.order_id.apply(lambda x: Convert_orderid(x))
df_result['order_id']=df_result.order_id.apply(lambda x: Convert_orderid(x))

df_order_1 = df_order.merge(df_group[group_used_cols], on='group_id')
# for order data
df_order_1['Order_Date']=df_order_1.order_date.apply(lambda x: Convert_Date(x))
df_order_1['Order_Tick']=df_order_1.order_date.apply(lambda x: Date2Ticks(x))
df_order_1['order_id']=df_order_1.order_id.apply(lambda x: Convert_orderid(x))
df_order_1['Source_1']= df_order_1.source_1.apply(lambda x: int(x[11:]))
df_order_1['Source_2']= df_order_1.source_2.apply(lambda x: int(x[11:]))
df_order_1['Unit']= df_order_1.unit.apply(lambda x: int(x[11:]))
df_order_1['Begin_Date']=pd.to_datetime(df_order_1['Begin_Date'])
df_order_1['Order_Date']=pd.to_datetime(df_order_1['Order_Date'])
df_order_1['PreDays']=(df_order_1['Begin_Date']-df_order_1['Order_Date']).dt.days
df_order_1['Begin_Date_Weekday']= df_order_1['Begin_Date'].dt.dayofweek
df_order_1['Order_Date_Weekday']= df_order_1['Order_Date'].dt.dayofweek
df_order_1['Return_Date_Weekday']= (df_order_1['Begin_Date'].dt.dayofweek+df_order_1['days'])%7
df_order_1['tick_diff'] = (df_order_1['Begin_Tick'] - df_order_1['Order_Tick'])/10000
df_order_1['price'] = df_order_1['price']/1000

order_used_columns=['order_id', 'group_id','tick_diff', 'Source_1', 'Source_2', 'Unit',
'people_amount', 'days', 'Area', 'SubLine', 'price',
'PreDays','Begin_Date_Weekday', 'Order_Date_Weekday', 'Return_Date_Weekday', 'name']

df_order_2=df_order_1[order_used_columns].merge(df_day_schedule[['group_id','title']], on='group_id')



C:\Users\Jess\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# train/test data
print("Got %d, %d training, testing data" % (len(df_train), len(df_result)))
df_train_1=df_train.merge(df_order_2,on='order_id')
df_result_1=df_result.merge(df_order_2,on='order_id')
print("Got %d, %d training, testing data" % (len(df_train_1), len(df_result_1)))

Got 297020, 99895 training, testing data
Got 297019, 99895 training, testing data


In [12]:
# train/test data
df_train_1=df_train.merge(df_order_2,on='order_id')
df_result_1=df_result.merge(df_order_2,on='order_id')

Y=df_train_1['deal_or_not'].values.tolist()
swX_tmp = df_train_1['title'].values.tolist()
Xid = df_train_1['order_id'].values.tolist()
del df_train_1['deal_or_not'] 
del df_train_1['title']
del df_train_1['group_id'] 
del df_train_1['order_id']
X = df_train_1.values.tolist()

rid = df_result_1['order_id'].values.tolist()
swrx = df_result_1['title'].values.tolist()
del df_result_1['deal_or_not']
del df_result_1['title']
del df_result_1['order_id']
del df_result_1['group_id']

rx = df_result_1.values.tolist()


sX, sY, Xid =np.asarray(X), np.asarray(Y), np.asarray(Xid)
rx,rid = np.asarray(rx), np.asarray(rid)
X,Y, swX=[],[], []
for i in range(len(sY)):
   # if (int(Xid[i])<=204000):
        X.append(sX[i,:])
        Y.append(sY[i])
        swX.append(swX_tmp[i])
X, Y = np.array(X), np.array(Y)


def text_to_index_array(p_new_dic, p_sen):  # 文本转为索引数字模式
    new_sentences = []
    for sen in p_sen:
        new_sen = []
        for word in str(sen):
            try:
                new_sen.append(p_new_dic[word])  # 单词转索引数字
            except:
                new_sen.append(0)  # 索引字典里没有的词转为数字0
        new_sentences.append(new_sen)

    return np.array(new_sentences)


wX = text_to_index_array(index_dict, swX)
wrx = text_to_index_array(index_dict, swrx)
wX = sequence.pad_sequences(wX, maxlen=200)
wrx = sequence.pad_sequences(wrx, maxlen=200)


# X=np.concatenate([X, wX], axis=1)
# rx=np.concatenate([rx, wrx], axis=1)

    
print(X.shape)

# np.save("data.npy", [X,Y,rx])
# [X,Y,rx] = np.load("data.npy")

(297019, 14)


In [13]:

folds = StratifiedKFold(n_splits= 10, shuffle=True)

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(rx.shape[0])
feature_importance_df = pd.DataFrame()


for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, Y)):

    train_x, train_y,train_id = X[train_idx,:], Y[train_idx], Xid[train_idx]
    valid_x, valid_y = X[valid_idx,:], Y[valid_idx]
    valid_id=Xid[valid_idx]

    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
    'nthread': 8, 'boosting_type': 'dart','objective': 'binary', 'metric': 'auc', 'learning_rate': 0.005, 'num_leaves': 70,
    'max_depth': 7, 'subsample': 1, 'feature_fraction': 0.4, 'colsample_bytree': 0.08, 'min_split_gain': 0.09,
    'min_child_weight': 9.5,
    'verbose': 1, 'devic': 'gpu',
    # parameters for dart
    'drop_rate':0.7, 'skip_drop':0.7, 'max_drop':5, 'uniform_drop':False, 'xgboost_dart_mode':True, 'drop_seed':5 }

    if n_fold >= 0:
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)
        bst = lgb.train(params, dtrain, num_boost_round=50000, valid_sets=[dval], early_stopping_rounds=5000, 
                        verbose_eval=5000)

        tmp_valid = bst.predict(valid_x, num_iteration=bst.best_iteration)
#         tmp_valid[valid_id>204000]=0
        oof_preds[valid_idx] = tmp_valid
        sub_preds += bst.predict(rx, num_iteration=bst.best_iteration) / folds.n_splits
        
#         sub_preds[rid>204000]=0

        # Make the feature importance dataframe
        gain = bst.feature_importance('gain')
        fold_importance_df = pd.DataFrame({'feature':bst.feature_name(),'split':bst.feature_importance('split'),
                                           'gain':100*gain/gain.sum(),  'fold':n_fold,}).sort_values('gain',ascending=False)

        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

        del bst, train_x, train_y, valid_x, valid_y
        
xid=list(Xid)
app_test = pd.read_csv('testing-set.csv', usecols=['order_id'])
preds = pd.DataFrame({"order_id":app_test['order_id'], "deal_or_not":sub_preds})
# create output sub-folder
preds.to_csv("output/lgb_dart_" + str(roc_auc_score(Y, oof_preds)) + ".csv", index=False)

Train Index: [     0      1      2 ... 297016 297017 297018] ,Val Index: [     9     10     11 ... 296966 296967 297006]
Training until validation scores don't improve for 5000 rounds.
[5000]	valid_0's auc: 0.691415
[10000]	valid_0's auc: 0.693339
[15000]	valid_0's auc: 0.693455
[20000]	valid_0's auc: 0.6935
[25000]	valid_0's auc: 0.69351
[30000]	valid_0's auc: 0.693515
Early stopping, best iteration is:
[28913]	valid_0's auc: 0.693517
Fold  1 AUC : 0.693489
Train Index: [     0      1      2 ... 297016 297017 297018] ,Val Index: [    12     13     14 ... 296992 297001 297013]
Training until validation scores don't improve for 5000 rounds.
[5000]	valid_0's auc: 0.682632
[10000]	valid_0's auc: 0.683283
Early stopping, best iteration is:
[9042]	valid_0's auc: 0.68334
Fold  2 AUC : 0.683363
Train Index: [     0      1      2 ... 297014 297015 297016] ,Val Index: [     8     16     18 ... 297012 297017 297018]
Training until validation scores don't improve for 5000 rounds.
[5000]	valid_0's